<h1>Task 1 - SQL: Problem statement</h1>

You are given two tables

TABLE **players**
 - player_id ( int unique)
 - group_id   (int)

TABLE **matches**
- match_id (int unique)
- first_player  ( player_id) (int)
- second_player (another player_id) (int)
- first_score (int)
- second_score (int)
	
Task: Write a query to get the winner in each group. 
- The results should include group id and player id. 
- Players and matches have 1 to many relationship. 
    - Meaning 1 player can play 0 to many matches and
    - In each match he can be the first or second player ( not the first and second at the same time)
- The player who scored the maximum number of points in each match is the winner of the match.
- The player who won the most matches within the group is the winner of the group.
- If more than one such player take the one with the lowestID.

Also if a player did not participate in any matches they are to be considered a winner. 

Assume that:
- first_player and second_player belong to the same group.
- groups are numbered with consecutive integers start from 1
- every players from table matches occurs in table players
- score is a value betwen 0 and 100000
- there are most 100 player_id and at most 100 match_id

In [2]:
import sqlite3

In [4]:
# Connect to an in-memory SQLite database
conn = sqlite3.connect(':memory:')
cur = conn.cursor()

In [5]:
# Create tables
cur.execute('CREATE TABLE players (player_id INT UNIQUE, group_id INT);')
cur.execute('CREATE TABLE matches (match_id INT UNIQUE, first_player INT, second_player INT, first_score INT, second_score INT);')

In [6]:
# Insert test data into tables
players_test_data = [
    (20, 2),
    (30, 1),
    (40, 3),
    (45, 1),
    (50, 2),
    (65, 1),
]

matches_test_data = [
    (1, 30, 45, 10, 12),
    (2, 20, 50, 5, 5),
    (13, 65, 45, 10, 10),
    (5, 30, 65, 3, 15),
    (42, 45, 65, 8, 4),
]

for player_data in players_test_data:
    cur.execute('INSERT INTO players VALUES (?, ?);', player_data)

for match_data in matches_test_data:
    cur.execute('INSERT INTO matches VALUES (?, ?, ?, ?, ?);', match_data)

conn.commit()

**Solution plan** 

-  First create the winner for each match name it winner_id
-  Note each match is unique.
-  Since all players from matches table belong to player_id
-  We join the winner for each match with the players. 
-  We need all players even those that we do not play any matches. (LEFT JOIN)
-  Therefore some winner id will be missing. Will need to deal with NULL values. (COALESCE)
-  Then we will count the winner player_ids within each group.
-  The winner id that appears the most in each group is the winner from each group.


In [9]:
solution_query = '''
WITH winners AS(
SELECT match_id,
    CASE 
        WHEN first_score > second_score THEN first_player
        WHEN first_score < second_score THEN second_player
        WHEN (first_score = second_score) AND (first_player < second_player) THEN first_player
        WHEN (first_score = second_score) AND (first_player > second_player) THEN second_player
    END AS winner_id
FROM matches 
         )
		 
SELECT group_id, winner_id
FROM(
	SELECT group_id, winner_id, 
	ROW_NUMBER() OVER( PARTITION BY group_id ORDER BY winner_id_times_won DESC) AS ranked_by_winner_id_times_won
	FROM(
		SELECT group_id, COALESCE(winner_id, p.player_id) as winner_id, COUNT(winner_id) as winner_id_times_won
		FROM players as p
		LEFT JOIN winners as w
		ON p.player_id = w.winner_id
		GROUP BY group_id, w.winner_id, p.player_id
		ORDER BY group_id) AS winners_by_group
	) winners_by_group_ranked
WHERE ranked_by_winner_id_times_won IN (1);
'''

In [10]:
# Execute the solution query and print the result
cur.execute(solution_query)
print("Group_id, Winner_id")
for row in cur.fetchall():
    print(row)

Group_id, Winner_id
(1, 45)
(2, 20)
(3, 40)


In [ ]:
# Close the connection
conn.close()

Final thougts and tips:
- Make sure you spend time understanding the problem. The actual problem description was more complicated with less info.
- Pay attention to detail. Especially by which variables to sort the results. That usually were code goes wrong when you run it.
- The platform had only the option for PostgreSQL.
- In general when you practicing for SQL interviews is best to practice on Postgres or MySQL
- I am using MS Server and there are quite a lot of differences between the two. 
- Some notable differences are in date functions and how MS Server handles CTEs.
- So be sure to not fall into that trap and expect all sql options.

- This task took me about 1h to complete. I was lucky it did not contain any date functions.
- Found the problem statement a bit difficult to understand.

 - **Overall I think the skill required to solve this is intermediate to advanced SQL**.